# Import Libraries

In [1]:
from tensorflow import keras
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from keras.datasets import mnist 
from tensorflow.keras.regularizers import l2
from keras import layers
from keras.utils import np_utils
from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

## Define the metrics
F measure is defined according to [this](https://deepai.org/machine-learning-glossary-and-terms/f-score)

In [3]:
# def f_measure(y_true, y_pred):
    
#     f = 2*precision_score(y_true, y_pred)*recall_score(y_true,y_pred)/(precision_score(y_true, y_pred) +
#                                                                      recall_score(y_true,y_pred) + K.epsilon())
#     return tf.convert_to_tensor(f) 
def precision_m(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall_m(y_true, y_pred):
    """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f_measure(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

## Data Preprocessing

In [4]:
# load the datasheet
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# transform the y data to categorical
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# x data normalisation
normalisation = False
if normalisation:
    x_train = x_train / np.max(x_train)
    x_test = x_test / np.max(x_test)

11501568/11490434 [==============================] - 0s 0us/step


## General Parameters

In [5]:
length_validation = 0.2
epochs = 1000
input_shape = [28, 28]
neurons_output = 10
batch_size = 256

total_neurons_hidden1 = [64, 128]
total_neurons_hidden2 = [256, 512]
total_alphas = [1e-1, 1e-3, 1e-6]
total_learning_rates = [1e-1, 1e-2, 1e-3]

## Build Model

In [6]:
def build_model(hp):
    
    neurons_hidden1 = hp.Choice('neurons_hidden1', values=total_neurons_hidden1)
    neurons_hidden2 = hp.Choice('neurons_hidden2', values=total_neurons_hidden2)
    alpha = keras.regularizers.L2(l2=hp.Choice('alpha', values=total_alphas))
    learning_rate = hp.Choice('learning_rate', values=total_learning_rates)
    initialiser = keras.initializers.HeNormal()
    
#     with tpu_strategy.scope():
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(neurons_hidden1, activation='relu', kernel_regularizer=alpha, kernel_initializer=initialiser))
    model.add(Dense(neurons_hidden2, activation='relu', kernel_regularizer=alpha, kernel_initializer=initialiser))
    model.add(Dense(neurons_output, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy', precision_m, recall_m, f_measure])
    
    return model
    

## Define the Tuner

In [7]:
tuner = kt.RandomSearch(
    build_model, objective=kt.Objective("f_measure", direction='max'))

early_stopping = EarlyStopping(
    monitor="loss",
    patience=200,
    restore_best_weights=True)

2022-09-20 13:11:02.347583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 13:11:02.448525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 13:11:02.449243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 13:11:02.450636: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

## Tune the Model

In [8]:
tuner.search(x_train, y_train,
             epochs=epochs,
             validation_split=length_validation,
             callbacks=[early_stopping])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
64                |?                 |neurons_hidden1
256               |?                 |neurons_hidden2
1e-06             |?                 |alpha
0.1               |?                 |learning_rate



2022-09-20 13:11:05.578531: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
1500/1500 [==============================] - 7s 3ms/step - loss: 457.7430 - accuracy: 0.1034 - precision_m: 0.0057 - recall_m: 5.4167e-04 - f_measure: 7.5335e-04 - val_loss: 2.3080 - val_accuracy: 0.0975 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f_measure: 0.0000e+00
Epoch 2/1000
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3139 - accuracy: 0.1061 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f_measure: 0.0000e+00 - val_loss: 2.3116 - val_accuracy: 0.0914 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f_measure: 0.0000e+00
Epoch 3/1000
1500/1500 [==============================] - 5s 3ms/step - loss: 2.3152 - accuracy: 0.1019 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f_measure: 0.0000e+00 - val_loss: 2.3108 - val_accuracy: 0.1035 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f_measure: 0.0000e+00
Epoch 4/1000
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3148 - acc

KeyboardInterrupt: 

## Get the optimal hyperparameters

In [ ]:
optimal_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

neurons_hidden1 = optimal_hyperparameters.get('neurons_hidden1')
neurons_hidden2 = optimal_hyperparameters.get('neurons_hidden2')
alpha = optimal_hyperparameters.get('alpha')
learning_rate = optimal_hyperparameters.get('learning_rate')

print(f'Best number of neurons in the first hidden layer: ' +str(neurons_hidden1))
print(f'Best number of neurons in the second hidden layer: ' +str(neurons_hidden2))
print(f'Best value for alpha: ' +str(alpha))
print(f'Best value for learning_rate: ' +str(learning_rate))